In [2]:
import pandas as pd

import difflib
from joblib import Parallel, delayed

import re

removelist = ".?!"
pattern = r'[^\w'+removelist+']'

def findRootSubject(subject_title):
    subject_root = subject_title.split(':')[1].strip()
    root_subjects = df_other_subjects[df_other_subjects['Subject'].str.contains(subject_root)]['Subject'].unique()
    
    matched_subjects = \
                [subject for subject in root_subjects if difflib.SequenceMatcher(None, a=subject, b=subject_root).ratio()>0.95]
    
    return [matched_subjects, subject_title]

In [3]:
df_restricted = pd.read_csv('all_results_process_stage_1a.csv', encoding='utf-8')

#Dedupe
df_restricted = df_restricted[df_restricted.duplicated(subset=['DateTime', 'From', 'To', 'Subject']) == False]

reply_subjects = df_restricted[(df_restricted['Subject'].str.contains('RE:')) | \
              (df_restricted['Subject'].str.contains('Re:')) | \
              (df_restricted['Subject'].str.contains('re:'))]['Subject'].values

fwd_subjects = df_restricted[(df_restricted['Subject'].str.contains('FW:')) | \
              (df_restricted['Subject'].str.contains('Fw:')) | \
              (df_restricted['Subject'].str.contains('fw:'))]['Subject'].values

thread_names = list(reply_subjects) + list(fwd_subjects)

In [4]:
len(thread_names)

86232

In [ ]:
if __name__ == '__main__':


    df_restricted = pd.read_csv('all_results_process_stage_1a.csv', encoding='utf-8')

    #Dedupe
    df_restricted = df_restricted[df_restricted.duplicated(subset=['DateTime', 'From', 'To', 'Subject']) == False]

    reply_subjects = df_restricted[(df_restricted['Subject'].str.contains('RE:')) | \
                  (df_restricted['Subject'].str.contains('Re:')) | \
                  (df_restricted['Subject'].str.contains('re:'))]['Subject'].values

    fwd_subjects = df_restricted[(df_restricted['Subject'].str.contains('FW:')) | \
                  (df_restricted['Subject'].str.contains('Fw:')) | \
                  (df_restricted['Subject'].str.contains('fw:'))]['Subject'].values

    thread_names = list(reply_subjects) + list(fwd_subjects)

    df_other_subjects = df_restricted[df_restricted['Subject'].isin(thread_names) == False]

    other_subjects = df_other_subjects['Subject'].unique()

    root_subjects = Parallel(n_jobs=-1, verbose=8)(delayed(findRootSubject)(subject) for subject in reply_subjects[:100])